In [24]:
import pandas as pd
from IPython.display import clear_output
from sqlalchemy.engine import create_engine
#from datasets import load_dataset
from openai import OpenAI
import io
import base64
import random
import requests
import torch
from PIL import Image
#from transformers import AutoProcessor,Qwen2_5_VLForConditionalGeneration
#from qwen_vl_utils import process_vision_info
import os
import pandas as pd
from sqlalchemy.engine import create_engine
#from transformers import AutoProcessor, BitsAndBytesConfig
import json
import yaml
import platform
from sqlalchemy import text
from IPython.display import display,clear_output
import time
import json
import pandas as pd
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

In [25]:
cnfig_file="/Users/bineshkumar/Documents/config.yaml"
def get_from_cnfg(key_path,file_path=cnfig_file):
    try:
        with open(file_path, 'r') as file:
            data = yaml.safe_load(file)

        keys = key_path.split('.')
        value = data
        for key in keys:
            value = value[key]
        return value

    except FileNotFoundError:
        print(f"File {file_path} not found")
    except yaml.YAMLError as e:
        print(f"YAML parsing error: {e}")
    except KeyError:
        print(f"Key path {key_path} not found")
    except Exception as e:
        print(f"Error: {e}")
    return None
os_name = platform.system()

In [26]:
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    from google.colab import userdata
    engine = create_engine(userdata.get('DB_URL'))
    gem_key=userdata.get('DB_URL')
    oai_key=userdata.get('DB_URL')
    source_folder='/content/drive/MyDrive/Health_Data/MIMIC_JPG_AVL/mimic-cxr-jpg/2.1.0/files/'
elif os_name == "Darwin":
    cnfig_file="/Users/bineshkumar/Documents/config.yaml"
    DB_URL = get_from_cnfg("cd_url",cnfig_file)
    gem_key=get_from_cnfg("gem_token",cnfig_file)
    oai_key=get_from_cnfg("oai_token",cnfig_file)
    source_folder='/Users/bineshkumar/Documents/mimic-cxr-jpg/2.1.0/files/'
elif os_name == "Linux":
    DB_URL = get_from_cnfg("cd_url",cnfig_file)
    gem_key=get_from_cnfg("gem_token",cnfig_file)
    oai_key=get_from_cnfg("oai_token",cnfig_file)
    source_folder=""
engine = create_engine(DB_URL)

In [27]:
def get_hourly_model_stats():
    query = text("""
    SELECT
        model_name,
        date_trunc('hour', created_at) as hour,
        COUNT(DISTINCT CONCAT(uid, '-', model_name, '-', question_id)) as unique_count
    FROM
        public.model_responses_r2
    GROUP BY
        model_name,
        date_trunc('hour', created_at)
    ORDER BY
        hour ASC,
        model_name
    """)

    with engine.connect() as conn:
        df = pd.read_sql_query(query, conn)
        return df

In [28]:
def visualize_stats(stats_df):
    # Create plots in a 3x1 layout (hourly, cumulative, heatmap)
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 18))

    # Convert timestamps to UTC to ensure consistency
    if 'hour' in stats_df.columns:
        if not pd.api.types.is_datetime64_dtype(stats_df['hour']):
            stats_df['hour'] = pd.to_datetime(stats_df['hour'])

        # If timestamps don't have timezone info, localize them to UTC
        if stats_df['hour'].dt.tz is None:
            stats_df['hour'] = stats_df['hour'].dt.tz_localize('UTC')

        # Create pivot table
        pivot_df = stats_df.pivot_table(
            index='hour',
            columns='model_name',
            values='unique_count',
            aggfunc='sum'
        ).fillna(0)

        # Plot 1: Hourly metrics
        if not pivot_df.empty:
            # Get the last 24 hours of data
            current_time_utc = pd.Timestamp.now(tz='UTC')
            day_ago_utc = current_time_utc - pd.Timedelta(days=1)
            recent_pivot = pivot_df[pivot_df.index >= day_ago_utc]

            if not recent_pivot.empty:
                recent_pivot.plot(kind='line', marker='o', ax=ax1)
                ax1.set_title('Hourly UIDs by Model - Last 24 Hours')
                ax1.set_xlabel('Hour')
                ax1.set_ylabel('Unique Combinations (uid-model-question)')
                ax1.grid(True)
            else:
                ax1.text(0.5, 0.5, "No data available for last 24 hours",
                         horizontalalignment='center', verticalalignment='center')

            # Plot 2: Cumulative metrics
            cumulative_df = pivot_df.cumsum()
            cumulative_df.plot(kind='line', ax=ax2)
            ax2.set_title('Cumulative UIDs by Model')
            ax2.set_xlabel('Hour')
            ax2.set_ylabel('Cumulative Unique Combinations')
            ax2.grid(True)

            # Plot 3: Heatmap of model activity (last 24 hours)
            if not recent_pivot.empty:
                sns.heatmap(recent_pivot.T, cmap="YlGnBu", annot=True, fmt="g", ax=ax3)
                ax3.set_title('Model Activity Heatmap - Last 24 Hours')
            else:
                ax3.text(0.5, 0.5, "No data available for heatmap",
                         horizontalalignment='center', verticalalignment='center')
        else:
            for ax in [ax1, ax2, ax3]:
                ax.text(0.5, 0.5, "No data available",
                        horizontalalignment='center', verticalalignment='center')
    else:
        for ax in [ax1, ax2, ax3]:
            ax.text(0.5, 0.5, "No data available",
                    horizontalalignment='center', verticalalignment='center')

    plt.tight_layout()
    return fig

In [1]:
# Function to show model stats table
def display_model_stats(stats_df):
    if not stats_df.empty:
        # Most recent hour stats
        most_recent_hour = stats_df['hour'].max()
        recent_stats = stats_df[stats_df['hour'] == most_recent_hour]

        # Calculate cumulative totals
        cumulative_stats = stats_df.pivot_table(
            index='model_name',
            values='unique_count',
            aggfunc='sum'
        ).reset_index().sort_values('unique_count', ascending=False)

        # Display stats for the most recent hour
        print(f"Model Stats for {most_recent_hour}:")
        display(recent_stats[['model_name', 'unique_count']].set_index('model_name').sort_values('unique_count', ascending=False))

        # Display cumulative stats
        print("\nCumulative Stats (All Time):")
        display(cumulative_stats.set_index('model_name'))

        # Calculate total UIDs
        total_recent = recent_stats['unique_count'].sum()
        total_cumulative = cumulative_stats['unique_count'].sum()
        print(f"Total unique combinations in the last hour: {total_recent}")
        print(f"Total unique combinations all time: {total_cumulative}")
    else:
        print("No data available")

# Create output widgets
output_text = widgets.Output()
output_plot = widgets.Output()

# Display the widgets
display(widgets.HTML("<h1>Real-time CockroachDB Model Monitoring</h1>"))
display(widgets.HBox([widgets.VBox([widgets.HTML("<h2>Stats</h2>"), output_text]),
                      widgets.VBox([widgets.HTML("<h2>Visualization</h2>"), output_plot])]))

# Function to update the dashboard
def update_dashboard(interval_seconds=60):
    try:
        stats_df = get_hourly_model_stats()

        # Update the text output
        with output_text:
            clear_output(wait=True)
            print(f"Last updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            display_model_stats(stats_df)

            # Also save to CSV
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            csv_filename = f"model_stats_{timestamp}.csv"
            stats_df.to_csv(csv_filename, index=False)
            print(f"Data saved to {csv_filename}")

        # Update the plot output
        with output_plot:
            clear_output(wait=True)
            fig = visualize_stats(stats_df)
            display(fig)
            plt.close(fig)

    except Exception as e:
        with output_text:
            clear_output(wait=True)
            print(f"Error: {e}")
            import traceback
            traceback.print_exc()

# Function to get cumulative trends for specific models
def get_model_cumulative_trend(model_names=None, days=30):
    try:
        if model_names:
            placeholders = ','.join([f"'{model}'" for model in model_names])
            where_clause = f"WHERE model_name IN ({placeholders})"
        else:
            where_clause = ""

        query = text(f"""
        SELECT
            model_name,
            date_trunc('hour', created_at) as hour,
            COUNT(DISTINCT CONCAT(uid, '-', model_name, '-', question_id)) as unique_count
        FROM
            public.model_responses_r2
        {where_clause}
        AND created_at >= NOW() - INTERVAL '{days} days'
        GROUP BY
            model_name,
            date_trunc('hour', created_at)
        ORDER BY
            hour
        """)

        with engine.connect() as conn:
            df = pd.read_sql_query(query, conn)

        # Make sure hour column is datetime type with timezone info
        df['hour'] = pd.to_datetime(df['hour'])
        if df['hour'].dt.tz is None:
            df['hour'] = df['hour'].dt.tz_localize('UTC')

        # Create both hourly and cumulative plots
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12))

        # Pivot and plot hourly data
        pivot_df = df.pivot_table(
            index='hour',
            columns='model_name',
            values='unique_count',
            aggfunc='sum'
        ).fillna(0)

        pivot_df.plot(kind='line', marker='o', ax=ax1)
        ax1.set_title(f'Hourly Metrics - Last {days} Days')
        ax1.set_xlabel('Hour')
        ax1.set_ylabel('Unique Combinations')
        ax1.grid(True)

        # Plot cumulative data
        cumulative_df = pivot_df.cumsum()
        cumulative_df.plot(kind='line', ax=ax2)
        ax2.set_title(f'Cumulative Metrics - Last {days} Days')
        ax2.set_xlabel('Hour')
        ax2.set_ylabel('Cumulative Unique Combinations')
        ax2.grid(True)

        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

# Initialize the dashboard
update_dashboard()

# Real-time monitoring function
def start_monitoring(interval_seconds=60):
    while True:
        update_dashboard(interval_seconds)
        time.sleep(interval_seconds)


NameError: name 'widgets' is not defined

In [30]:
from IPython.lib import backgroundjobs
jobs = backgroundjobs.BackgroundJobManager()
jobs.new(start_monitoring, 60)  # Updates every 60 seconds

<BackgroundJob #0: <function start_monitoring at 0x3cab2aac0>>